
Aufbau der Seite 

Fachgebiete
    https://arxiv.org/ <br>
    /archive/<?:str>

    Jahr
        /year/<astro-ph:str>/<number:int>
        https://arxiv.org/list/<astro-ph:str>/<2301:int>

        Paper pages
            e.g.: <br>
            https://arxiv.org/list/astro-ph/2301?skip=25&show=25

        Abstract:
            https://arxiv.org/abs/2301.11140


    
    Methodik
    request arxiv.org 
    parse themes

        request html
        parse year
        for year in years:
            request
            parse month
            for month in month:
                request
                parse years
                for page in pages
                    request
                    parse paper
                    for paper in papers
                        request
                        parse paper
    

In [ ]:
import sqlite3
from database import Database
import configuration
import datetime
import requests
import re

# create sqlite3 database
path = "./../data/"                                     # change if needed
#db_name = "cs.db"                                      # change if needed
db_name = configuration.DATABASE_NAME                   # change if needed


# empty for crawling all
# for white_list and ignore_list watch below (cell 4)
only_themes_to_crawl:list[str] = []                     # change if needed 

db = "{}{}".format(path, db_name)
print(db)

# to create new database
#Database().createDatabase(db)                          # comment out if new db needed

conn = sqlite3.connect(db)

cursor = conn.cursor()
cursor.execute('SELECT COUNT(*) FROM paper')
count = cursor.fetchone()[0]
print("Number of rows in 'paper':", count)

domain = "https://arxiv.org/"

In [ ]:
document = requests.get(domain).content
document = (str) (document)

In [ ]:
themes = []

positions = []
positions=([m.end() for m in re.finditer("/archive/",document)])

for start in positions:
    end = document.find('"', start)
    themes.append((document[start:end]))

# cs (computer science) is a special case does not get catched by choosen method
themes.append("cs")

print(themes)

In [ ]:
# use numbers as string
black_list = [
        {
            'theme' : '',
            'ignore_theme' : False,
            'year' : [],
            'month' : []
        }
]

white_list = [
        {
            'theme' : '',
            'year' : [],
            'month' : []
        }
]

if len(only_themes_to_crawl) > 0:
    themes = only_themes_to_crawl

In [ ]:
print("Black listed:")
print(black_list)

print("\n\nWhite listed:")
print(white_list)
print()
print(themes)

In [ ]:

print("Starting crawling: {}".format(themes))
for theme in themes:
    skip_theme = False
    for black_entry in black_list:
        if theme == black_entry['theme'] and black_entry['ignore_theme'] == True:
            skip_theme = True
            break
    for white_entry in white_list:
        if theme == white_entry['theme']:# and white_entry['ignore_theme'] == True:
            skip_theme = False
            break
    if skip_theme == True:
        skip_theme = False
        continue
    print("Starting crawling: {}".format(theme))

    url_theme = domain + "archive/" + theme
    print(url_theme)
    document = requests.get(url_theme).content
    document = (str) (document)
    
    url_year = "year/" + theme + "/"
    positions = []
    positions=([m.end() for m in re.finditer(url_year,document)])
    print(url_year)
    
    years = []
        
    for start in positions:
        end = document.find('"', start)
        years.append((document[start:end]))
    print("Found years: {}".format(years))
    print()


    for year in years:
        
        skip_year = False
        for black_entry in black_list:
            if theme == black_entry['theme'] and year in black_entry['year']:
                skip_year = True
                break
        for white_entry in white_list:
            if theme == white_entry['theme'] and year in white_entry['year']:
                skip_year = False
                break
        if skip_year == True:
            skip_year = False
            continue
        print("Starting crawling: {}".format(year))


        url_month = domain + url_year + year
        print(url_month)
        document = requests.get(url_month).content
        document = (str) (document)
        
        url_month = "/list/" + theme + "/"
        print(url_month)
        positions = []
        positions=([m.end() for m in re.finditer(url_month,document)])
        
        months = []
            
        for start in positions:
            end = document.find('"', start)
            value = document[start:end]
            if not value.isdigit():
                continue
            if value in months:
                continue
            months.append((value))
        print("Found month in year {}: {}".format(year, months))
        
        
        findString = "<small>[ total of "
        for month in months:
            
            skip_month = False
            for black_entry in black_list:
                if theme == black_entry['theme'] and month in black_entry['month']:
                    skip_month = True
                    break
            for white_entry in white_list:
                if theme == white_entry['theme'] and month in white_entry['month']:
                    skip_month = False
                    break
            if skip_month == True:
                skip_month = False
                continue
            print("Starting crawling: {}".format(month))

            
            url_month = domain + "list/" + theme + "/" + month
            print(url_month)
            document = requests.get(url_month).content
            document = (str) (document)
            filter = "<small>[ total of "

            start_position = document.find(filter)
            start_position = start_position + len(filter)

            end_position = document.find("entries",start_position)
            number_of_papers = (int)(document[start_position:end_position])
            print("number of papers: {}".format(number_of_papers))
            #print(url_month)
 
            number_of_papers_per_page = 100
            for page in range(0, number_of_papers, number_of_papers_per_page):
                url_page = url_month + "?skip={}&show={}".format(page,number_of_papers_per_page)
                #print(url_page)
                document = requests.get(url_page).content
                document = (str) (document)
                filter = '<a href="/abs/'
                papers_url = domain + "abs/"
                positions = []
                positions=([m.end() for m in re.finditer(filter,document)])
                papers = []
                    
                for start in positions:
                    end = document.find('"', start)
                    value = document[start:end]
                    
                    papers.append((value))

                #print(papers)


                url = domain + "abs/"
                for paper in papers:
                    paper_url = url + paper
                    #print(paper_url)
                    cursor.execute('SELECT COUNT(*) FROM paper WHERE url=?', (paper_url,))
                    result = cursor.fetchone()

                    if result[0] == 0:
                        document = requests.get(paper_url).content
                        document = (str) (document)

                        cursor.execute("INSERT INTO paper (url, theme, timestamp, content) VALUES (?, ?, ?, ?)", (paper_url, theme, datetime.datetime.now(), document))
                        conn.commit()
                        #print(paper_url)


In [ ]:
cursor.execute('SELECT COUNT(*) FROM paper')
count = cursor.fetchone()[0]
print("Number of rows in 'paper':", count)

In [ ]:
cursor.execute('SELECT * FROM paper')
document = cursor.fetchall()[0]
print(document)

In [ ]:
cursor.close()